# **Import Neccessary Libraies & Functions**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Install Required Libraries ---
# Install the necessary libraries for dataset handling and hyperparameter optimization
!pip install datasets
!pip install optuna

In [ ]:
## --- Import Libraries ---
# Import all required libraries for data processing, modeling, and evaluation
from google.colab import drive  # For accessing Google Drive
import re
import os
import shutil
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    confusion_matrix,
    classification_report,
    make_scorer,
    f1_score
)
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoConfig
import kagglehub
from kagglehub import KaggleDatasetAdapter
import matplotlib.pyplot as plt
import seaborn as sns


# --- Mount Google Drive ---
# Mount Google Drive to access datasets or save results
drive.mount('/content/drive')



## **Imoprt Custom Functions**

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/YELP-Sentiment_analysis')

In [ ]:
# Import reusable functions from utility.py
from utility import (
    load_dataset,
    perform_eda,
    preprocess_yelp_reviews,
    advanced_text_preprocessing,
    prepare_datasets,
    create_model_config,
    compute_metrics,
    evaluate_model_on_test,
    save_trained_model_and_tokenizer
)

In [6]:
def main(perform_eda_flag=False, visualize_eda=False):
    # Load the dataset
    file_path = "Final.csv"
    columns = ["text", "stars"]
    review_df = load_dataset(file_path, columns)

    # Preprocess the data
    review_df = preprocess_yelp_reviews(review_df)
    review_df['processed_text'] = review_df['text'].apply(advanced_text_preprocessing)

    # Prepare datasets (use the entire dataset)
    train_dataset, val_dataset, test_dataset, tokenizer = prepare_datasets(review_df)

    # Load the best hyperparameters (replace with actual values)
    best_hyperparameters = {
        "learning_rate": 3e-5,
        "per_device_train_batch_size": 16,
        "num_train_epochs": 5
    }

    # Define training arguments with the best hyperparameters
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,  # From hyperparameter search
        learning_rate=7.91e-5,  # From hyperparameter search
        per_device_train_batch_size=32,  # From hyperparameter search
        seed = 5, # From hyperparameter search
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=50,
        report_to="wandb",  # Log metrics to Wandb
        load_best_model_at_end=True
    )

    # Initialize the model
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        config=create_model_config()
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the test dataset
    test_results = evaluate_model_on_test(trainer, test_dataset)

    # Save the trained model
    save_trained_model_and_tokenizer(trainer, tokenizer, save_path="/content/drive/MyDrive/YELP-Sentiment_analysis")



# Run the main function
if __name__ == "__main__":
    main(perform_eda_flag=True, visualize_eda=True)  # Set to True if you want to perform EDA

100%|██████████| 1.33G/1.33G [01:01<00:00, 23.3MB/s]

Extracting zip of Final.csv...



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fitsumbahlbi (fitsumbahlbi-lighthouse-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.511000,0.512808,0.775636,0.773667,0.773073,0.775636
2,0.461000,0.495084,0.782643,0.781248,0.780719,0.782643


Evaluating the model on the test dataset...


Test Evaluation Results:
eval_loss: 0.4936
eval_accuracy: 0.7840
eval_f1: 0.7825
eval_precision: 0.7819
eval_recall: 0.7840
eval_runtime: 423.0232
eval_samples_per_second: 254.7330
eval_steps_per_second: 31.8420
epoch: 2.0000
Saving the trained model and tokenizer to /content/drive/MyDrive/YELP-Sentiment_analysis_20250330_025931...
Model saved successfully!
Tokenizer saved successfully!


In [12]:
# 1- Authenticate with Hugging Face:
from huggingface_hub import notebook_login
notebook_login()


In [8]:
# 2- Install Git Large File Storage (Git LFS):
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [10]:
!pip install transformers huggingface_hub


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from huggingface_hub import HfApi

# Load the model and tokenizer
model_dir = "/content/drive/MyDrive/YELP-Sentiment_analysis_20250330_025931"
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Push the model and tokenizer to your Hugging Face repository
model.push_to_hub("fitsblb/YelpReviewsAnalyzer")
tokenizer.push_to_hub("fitsblb/YelpReviewsAnalyzer")



README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fitsblb/YelpReviewsAnalyzer/commit/7ce87b29e0528dc5fdb8458ad28d4da60498c393', commit_message='Upload tokenizer', commit_description='', oid='7ce87b29e0528dc5fdb8458ad28d4da60498c393', pr_url=None, repo_url=RepoUrl('https://huggingface.co/fitsblb/YelpReviewsAnalyzer', endpoint='https://huggingface.co', repo_type='model', repo_id='fitsblb/YelpReviewsAnalyzer'), pr_revision=None, pr_num=None)

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define your Hugging Face repository name
repo_name = "fitsblb/YelpReviewsAnalyzer"

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(repo_name)
tokenizer = AutoTokenizer.from_pretrained(repo_name)


config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [16]:
import torch

# Sample review for testing
sample_text = "The food was absolutely amazing and the service was top-notch!"

# Tokenize the input
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True)

# Generate predictions
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Interpret the prediction (assuming your model is sentiment analysis)
sentiment = "Positive" if predictions.item() == 1 else "Negative"
print(f"Predicted sentiment: {sentiment}")


Predicted sentiment: Negative


In [19]:
from transformers import pipeline

# Load model from Hugging Face Hub
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="fitsblb/YelpReviewsAnalyzer"  # Replace with your Hugging Face repo name
)

# Define a custom function to map predicted labels to sentiments
def map_sentiment_label(label):
    if label == "LABEL_0":
        return "Negative"
    elif label == "LABEL_1":
        return "Neutral"
    elif label == "LABEL_2":
        return "Positive"
    else:
        return "Unknown"

# Test the model with a sample input
sample_text = "The food was absolutely amazing and the service was top-notch!"
output = sentiment_pipeline(sample_text)

# Extract the label and map it to your custom sentiment
predicted_label = output[0]['label']  # Use the label returned from the pipeline
sentiment = map_sentiment_label(predicted_label)

print(f"Review: {sample_text}\nPredicted sentiment: {sentiment} (Label: {predicted_label})")


Device set to use cuda:0


Review: The food was absolutely amazing and the service was top-notch!
Predicted sentiment: Positive (Label: LABEL_2)
